##  Python Ödev:

•	Kullanıcıdan alınan İSİM, SOYİSİM, TC, DOĞUM TARİHİ bilgilerinin gerçek bir tüzel kişiye mi ait olduğunu kontrol eden fonksiyon yazınız. (bu kontrol için edevlet’in yazmış olduğu web servisi kullanın)

•	Bu inputlar fonksiyona dışarıdan (kullanıcı yazma işlemini yapacak) parametre olarak gönderilecektir.

•	Fonksiyon sonucu True veya False dönmeli. Fonksiyon sonucundan dönen değer doğru ise, gelen veri bir önceki ödevde verilen veritabanı bilgileriyle tabloya eklenmelidir. Dönen değer False ise, ekrana bilgi mesajı dönülmelidir. ( mysql veritabanında isim_soyisim_ikinciodev adında tablo oluşturup bu tabloya verileri ekleyin)
	

In [1]:
#Verileri alalım
def get_info():
    name = input("İsim(Büyük harfler): ")
    surname = input("Soyisim(Büyük harfler): ")
    tc = int(input("Tc: "))
    birth = int(input("Doğum tarihi(Yıl): "))
    args={
    "Ad":name,
    "Soyad":surname,
    "TCKimlikNo":tc,
    "DogumYili":birth
    }
    return args
    

In [9]:
import mysql.connector
from mysql.connector.errors import Error
def connectDB():
    try:
        cnx = mysql.connector.connect(user='root', password='istdsa',
                                  host='35.202.76.28',
                                  database='istdsa_de_new')
        return cnx
    except Error as e:
        print(e)
        return False
def createTable(tableName):
    cnx = connectDB()
    if cnx:
        mycursor = cnx.cursor()
        mycursor.execute("CREATE TABLE IF NOT EXISTS " + tableName + " (NAME VARCHAR(255),SURNAME VARCHAR(255),TC INT,BIRTH INT)")
        print("Tablo oluşturuldu")
        cnx.close()
    else:
        print("Hata")
def deleteTable(tableName):
    cnx = connectDB()
    if cnx:
        mycursor = cnx.cursor()
        mycursor.execute("DROP TABLE IF EXISTS "+tableName)
        print("{} silindi".format(tableName))
        cnx.close()
def listTables():
    cnx = connectDB()
    if cnx:
        mycursor = cnx.cursor()
        mycursor.execute("SHOW TABLES")
        table_list = []
        for table in mycursor:
            table_list.append(table)
        cnx.close()
        return table_list
    
def checkTable(tablename):
    cnx = connectDB()
    if cnx:
        mycursor = cnx.cursor()
        mycursor.execute("SHOW TABLES LIKE '"+tablename+"'")
        return True
    else:
        return False
    
def addValue(tablename,args):
    #Tablo var mı yok mu kontrol et
    if checkTable(tablename):
        cnx = connectDB()
        if cnx:
            mycursor = cnx.cursor()
            insert_stmt = (
            "INSERT INTO "+table_name+" (NAME,SURNAME,TC,BIRTH)"
            "VALUES (%s, %s, %s, %s)")
            mycursor.execute(insert_stmt,args)
            cnx.commit()
            mycursor.close()
            print("Veriler eklendi")
    else:
        print("Tablo bulunamadı")
def showTable(tableName):
    cnx = connectDB()
    if cnx:
        mycursor = cnx.cursor()
        mycursor.execute("SELECT * FROM "+table_name)
        return mycursor.fetchall()

In [121]:
#Rest bağlantısı için lib indirelim
!pip install suds-jurko

In [3]:
from suds.client import Client
WSDL_URL="https://tckimlik.nvi.gov.tr/Service/KPSPublic.asmx?WSDL"
client=Client(WSDL_URL)

In [10]:
#Kişinin olup olmadığını check eden func yazalım
def dogrula(args):
    if client.service.TCKimlikNoDogrula(**args):
        return True
        
    else:
        return False

In [25]:
#Örnek verilerle user_data oluşturalım
user_data=get_info()

İsim(Büyük harfler): ECEM
Soyisim(Büyük harfler): PANGAL
Tc: 38173221748
Doğum tarihi(Yıl): 1997


In [26]:
#Deneme için TC no yu eksik yazdım ve false döndü
dogrula(user_data)

True

In [13]:
user_data

{'Ad': 'İSMAİL', 'Soyad': 'ÇAĞILCI', 'TCKimlikNo': 23, 'DogumYili': 1996}

In [15]:
#True döndüğünce veriyi db ye yazıcaz ama TC mi yazmak istemediğimden bu şekilde deneyelim"
#İlk önce tablomuz var mı yok mu check edelim
table_name = "ismail_çağılcı_ikinciödev"
checkTable(table_name)

True

In [16]:
#Var olan tablomuzu silelim
deleteTable(table_name)

ismail_çağılcı_ikinciödev silindi


In [17]:
#User datamızın elemanlarını listeye atalım
data_list = []
for key,value in user_data.items():
    data_list.append(value)

In [18]:
#Tablo oluşturalım

createTable(table_name)

Tablo oluşturuldu


In [19]:
#User data verilerimizi yeni oluşturduğumuz tabloya ekleyelim
addValue(table_name,tuple(data_list))

Veriler eklendi


In [20]:
#Verileri kontrol edelim
table_data = showTable(table_name)

In [22]:
#Eklenen verilerimize bakalım
table_data

[('İSMAİL', 'ÇAĞILCI', 23, 1996)]